In [2]:
# Dependencies
import requests
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs 
from splinter import Browser

In [3]:
#basic variables
url = "https://mars.nasa.gov/news/"
response = requests.get(url)
response

<Response [200]>

In [4]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')

In [6]:
#sanity check
print(soup.prettify())

uot;:&quot;&quot;},&quot;highlight_current&quot;:{&quot;highlight&quot;:true,&quot;current_id&quot;:83,&quot;parent_ids&quot;:[]},&quot;search_submit&quot;:&quot;/search/&quot;}">
         </nav>
        </div>
       </div>
      </div>
     </header>
    </div>
    <div id="sticky_nav_spacer">
    </div>
    <div id="page">
     <div class="page_cover">
     </div>
     <!-- title to go in the page_header -->
     <div class="header_mask">
      <section class="content_page module">
      </section>
     </div>
     <div class="grid_list_page module content_page">
      <div class="grid_layout">
       <article>
        <header id="page_header">
        </header>
        <div class="react_grid_list grid_list_container" data-react-cache-id="GridListPage-0" data-react-class="GridListPage" data-react-props='{"left_column":false,"class_name":"","default_view":"list_view","model":"news_items","view_toggle":false,"search":"true","list_item":"News","title":"News","categories":["19,165,184,2

In [7]:
#Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.
newsTitle = soup.find('div', class_='content_title').text
newsTitle = str(newsTitle)
print(newsTitle)




Alabama High School Student Names NASA's Mars Helicopter




In [8]:
newsP = soup.find('div', class_='rollover_description_inner').text
newsP = str(newsP)
print(newsP)


Vaneeza Rupani's essay was chosen as the name for the small spacecraft, which will mark NASA's first attempt at powered flight on another planet.



In [48]:
#use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url
#make sure to find the image url to the full size .jpg image
#make sure to save a complete url string for this image
jplNasaUrl = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jplNasaUrl)
html = browser.html
jplSoup = bs(html, 'html.parser')
featuredImageDiv = jplSoup.find('article', class_='carousel_item')
#print(featuredImageDiv)
articleStyle = featuredImageDiv.get('style')
#get url from within style entry
extractedURL = articleStyle[articleStyle.find("('")+2:articleStyle.find("')")]
#create full url
featuredMarsImgURL = "http://www.jpl.nasa.gov" + extractedURL
print("Image URL for featured image on Mars")
print(featuredMarsImgURL)
#visit img
browser.visit(featuredMarsImgURL)


Image URL for featured image on Mars
http://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16092-1920x1200.jpg


In [69]:
#scrape the latest Mars weather tweet from the page. Save the tweet text weather report as a variable called mars_weather
tweetUrl = 'https://twitter.com/marswxreport?lang=en'
browser.visit(tweetUrl)

browser.is_element_present_by_css("section.css-1dbjc4n > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > article:nth-child(1) > div:nth-child(1) > div:nth-child(2)",wait_time=10)

tweets = browser.find_by_css("#react-root > div > div > div.css-1dbjc4n.r-13qz1uu.r-417010 > main > div > div > div > div > div > div > div > div > div:nth-child(3) > section > div > div > div:nth-child(1) > div > div > div > div > article > div > div.css-1dbjc4n.r-18u37iz.r-thb0q2 > div.css-1dbjc4n.r-1iusvr4.r-16y2uox.r-1777fci.r-5f2r5o.r-1mi0q7o > div:nth-child(2) > div:nth-child(1) > div > span")

marsTweet = tweets[0].text
print(marsTweet)

InSight sol 506 (2020-04-29) low -93.5ºC (-136.2ºF) high -3.7ºC (25.3ºF)
winds from the WNW at 4.5 m/s (10.1 mph) gusting to 15.6 m/s (34.8 mph)
pressure at 6.80 hPa


In [83]:
#use pandas to convert the data to a HTML table str
marsFacts = requests.get("https://space-facts.com/mars/")
marsTable = pd.read_html(marsFacts.text)
marsDf = marsTable[0]
marsDf.set_index(0, inplace=True)
marsDf

,1
0,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [84]:
marsDfHtml = marsDf.to_html()
marsDfHtml

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>1</th>\n    </tr>\n    <tr>\n      <th>0</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n<

In [ ]:
#obtain USGS Astrogeology site to obtain high resolution images for each of Mar's hemispheres
#save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name
#append the dict w the img url str and the hemisphere title to a list, one dict per hemisphere

In [ ]:
#MongoDB & Flask Application